# Computer Vision, Lab 05: Cameras and lenses

Today we'll learn about cameras, lenses, and how to calibrate a camera.
Some references for this material:

 - https://docs.opencv.org/master/d4/d94/tutorial_camera_calibration.html
 - Halcon Manual
 - https://www.baslerweb.com/
 - https://www.baslerweb.com/en/products/tools/lens-selector/
 - https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html
 - https://learnopencv.com/camera-calibration-using-opencv/

## Camera Systems

The camera system is an important module in any machine vision system. The camera sytem integrates with an overall machine vision system as follows.

<img src="img/lab04-1.png" width="600"/>

The important parameters in a camera system are:

 1. Resolution: size of relevant aspect of object of interest.
 2. Field of View (FOV): The area the camera captures. HFOV and VFOV indicate the horizontal and vertical FOVs.
 3. Working Distance (WD): Distance between the camera and object. Fixed in an industrial system or variable in other applications.
 4. Sensor Size: Size of the CCD or CMOS image sensor chip on the digital camera's circuit board.
 5. Depth of Field (DOF): Distance between the nearest and farthest objects that are in focus.
 6. Image Resolution: Size of image, $(w \times h)$. Standard sizes such as 1920$\times$1080 HD are referred to by the vertical resolution,
    e.g. 1080p.
 7. Pixel: Grayscale, RGB, multispectral, etc. For a fixed working distance, a pixel's backprojection has a fixed size, e.g., 2 mm per pixel.
 8. Object Resolution: size of object of interest when imaged at the minimum size (e.g., faces at 20$\times$20 or
    humans at 64$\times$128).
 9. Focal Length: Distance between the image plane and the optical center of the camera, measured in pixels or mm.
 10. Magnification: Ratio between sensor size and FOV.

<img src="img/lab04-2.png" width="800"/>

DOF varies depending on aperture:

<img src="img/lab04-3.png" width="400"/>
 
### Image Sensors
 
When an image is captured by the image sensor, the state of each photoreceptor
is saved then transferred to host memory. Image sensors in digital cameras and
industrial sensors may be one of two types:

 - CCDs convert photoreceptor charges to analog voltages then digitized off-chip
 - CMOS sensors have separate digitization circuitry for every pixel

Early fabrication techniques in the 1970s and 1980s were more suitable for CCDs, so available CCD sensors delivered much higher quality images and therefore
dominated machine vision. In the 1990s, mobile phone commoditization started pushing CMOS technology, and now the available CMOS sensors are equal to or greater
than CCDs in quality and much cheaper, though CCDs still deliver higher quality NIR imaging than CMOS sensors.

Image sensor size is the main way to adjust the quality of the image output. Early on, most sensors had 4:3 aspect ratios, but now HD resolutions with 16:9
ratios are becoming more common.
 
<img src="img/lab04-4.png" width="500"/>

Depending on the size and density of the photoreceptors on the image sensor, we get different image resolutions.

<img src="img/lab04-5.jpg" width="400"/>

### How to know if resolution is suitable for your work?

The International Standards Organization has standard 12233, which
defines various standardized charts that can be used to assess the
quality of an image. Here is the "legacy" (pre-2000) chart:

<img src="img/lab04-6.jpg" width="800"/>

You can purchase charts from the ISO or print one yourself. See
[Stephen Westin's Cornell Graphics web page on the ISO resolution chart](https://www.graphics.cornell.edu/~westin/misc/res-chart.html)
for more detail.

A rough idea can be obtained by considering the physical size of the smallest detail you want to analyze, considering how accurately
you want to analyze it (the minimum number of pixels that should span the detail), then using that to calculate the necessary image
resolution, lens focal length, and working distance. For example, suppose

1. You want to detect cracks in a surface object that are a minimum 0.5. mm wide.
2. You believe your crack detection algorithm requires at least 4 pixels of width
   to ensure accurate detection (8 pixels per mm).
3. You like the image quality and price of a 2 MP HD camera with a 50 degree HFOV.

Since object size in pixels is $fX/Z$, we can write $$4 \text{pixels} = f \text{pixels} \times 0.5 \text{mm} / D.$$
To get $f$, we know that $$\tan^{-1}(25^\circ) = 960 / f,$$ so $f = 2333.3$ and $D = 0.2916$ m. If a 29 cm working distance
is OK for our application, the last step would be to ensure that the camera can focus on objects at that distance and make sure
the DOF around a focus of 29 cm is sufficient.

### Frame rate

Another parameter that is important in most applications is the camera's frame rate. It is measured in frames per second.
When you are imaging objects in motion, you should consider the maximum object velocity at the minimum working distance. That
will tell you the maximum object displacement to expect per frame. For object tracking, you would want to ensure a fast enough
frame rate that successive images of the moving object overlap sufficiently to have high confidence when associating the object
appearance in a new frame with the object's appearance in the previous frame.

Three other factors are related to frame rate: exposure time, shutter type, and interlacing.

A short exposure time will minimize motion blur but will also decrease contrast. A longer exposure time will allow more light
to reach the sensor, improving contrast but also increasing motion blur.

The shutter can be global (CCD sensors and some CMOS sensors) or rolling (most CMOS sensors). A global shutter captures the entire
image simultaneously, while a rolling shutter exposes sensor elements sequentially. If the shutter is fast enough relative to object motion,
the shutter type is unimprotant. But if object motion (or camera motion) is high, a global shutter is preferred.

Interlacing means that multiple passes are required to piece together the image. For example, even-numbered rows may be imaged on the
first pass and odd-numbered rows may be imaged on the second pass. Interlacing is not common on modern sensors.


## Understanding Lens Specifications

Lens specifications have two important measurements: Focal Length and Aperture (F-Stop)

### Aperture (F-Stop)

Aperture (called "F-Stop" in commercial camera specifications)
indicates the width of the opening allowing light into the lens.
Aperture (A) is measured relative to focal length:

$$A = \frac{f}{S},$$

where $S$ is the "aperture scale." Smaller aperture scales mean more light than
larger aperture scales.

<img src="img/lab04-7.jpg" width="600"/>

When illumination is low, you may need larger apertures (smaller aperture scale numbers):

<img src="img/lab04-9.jpg" width="600"/>

And conversely, when light is especially bright,
you may need smaller apertures. Aperture has interesting effects
on especially bright light sources like the sun:

<img src="img/lab04-8.jpg" width="500"/>

But more importantly, when you have good illumination, you can afford to reduce the aperture
(increase the aperture number), which has the benefit of giving you a larger DOF:

<img src="img/lab04-10.jpg" width="500"/>

Obviously, you would like to have a large DOF for a mobile robot, so you might want a very small
aperture, but on the other hand, you also want a wide field of view (shorter focal length),
which reduces your relative aperture size.

### Focal length

Focal length is the distance between the image sensor and the center of projection. In camera
specifications, it is measured in mm, whereas in machine vision calculations, it is measured in pixels.
Longer focal lengths give more zoom and smaller FOV.

<img src="img/lab04-11.png" width="1000"/>

A small focal length is great for imaging a wide field of view, but spreading the image out so much
also typically requires a more convex lens, which increases the radial distortion.

<img src="img/lab04-12.png" width="800"/>

<img src="img/lab04-13.gif" width="400"/>

### Planar Perspective Correction

When your camera's principal axis is not orthogonal to the surface you're imaging, you will experience perspective distortion. As
we've seen in class, if the object is truly planar, we can correct the image using a 4-point homography.

### Radial Distortion

The word "distortion" might be used to describe any kind of warping of an object due to depth, non-orthogonal principal axis, or
"bending" of straight lines. In computer vision, we normally reserve the word "distortion" to refer to non-pinhole effects arising
from the use of a lens to achieve the desired projection onto the image sensor. Non-pinhole distortion is either *radial* (warping
along a vector emanting from a center of distortion), which is very common, especially with wide angle lenses, and *tangential*
(warping orthogonal to the radial direction), which is less common and normally arises due to imprecise lens mounting. Getting precise
measurements from a camera system requires either taking distortion into account or eliminating it. Distortion
can be eliminated using optical techniques (using compound lenses and other hardware techniques), but this is expensive and possibly bulky.
A more practical solution to eliminate distortion is to model the distortion function of your camera system then either use that model
in your calculations or rectify every image before further processing.

The modeling process is called *spatial calibration* and uses a known calibration pattern:

<img src="img/lab04-14.png" width="800"/>

## Geometric Camera Parameters

Knowing the camera fundamentals above, we can use a calibration
process to calculate camera parameters in geometric form. We'll assume
the camera's optical center as the origin of a 3D coordinate reference frame
and the principal point of the camera as the point in the image at which the
camera's principal axis and image plane intersect.

<img src="img/lab04-15.gif" width="600"/>

### Types of parameters

There are two types of parameters to calculate to reconstruct the 3D structure of a scene from the pixel coordinates of image points:

1. Extrinsic camera parameters: Parameters defining location and orientation of the camera reference frame in a world reference frame.
2. Intrinsic camera parameters: Parameters linking 3D points in the camera reference frame to image points.

<img src="img/lab04-17.png" width="400"/>

<img src="img/lab04-16.png" width="600"/>

As the figure shows, we can express a camera coordinate point in terms of a world coordinate using the intrinsic and extrinsic matrices as:

\begin{equation}
\mathbf{x} = \begin{bmatrix}
x \\
y \\
1
\end{bmatrix} \propto \mathtt{K} \mathbf{X}_c = \mathtt{K} \begin{bmatrix} \mathtt{R} \mid \mathbf{t} \end{bmatrix} \begin{bmatrix}
X \\
Y \\
Z \\
1
\end{bmatrix}
\end{equation}

Where
 - $\mathbf{X}_c$ is a 3D camera point
 - $\mathbf{X}_w = \begin{bmatrix} X & Y & Z & 1 \end{bmatrix}^\top$ is a 3D world point in homogeneous form
 - $\mathtt{K}$ is the intrinsic calibration matrix


### Extrinsic Camera Parameters

The parameters $\mathtt{R}$ and $\mathbf{t}$ mean

1. The origin of the world coordiante system in the camera reference frame.
2. The rotation matrix that brings the corresponding axes of the two frames into alignment (i.e., onto each other).

Using the extrinsic camera parameters, we can find the relation between the coordinates of a point P in world ($P_w$) and image plane ($P_{im}$) coordinates:

$$
\mathbf{x} \propto \mathtt{K}\mathtt{R}(\tilde{\mathbf{X}}_w - \mathbf{C}) = \mathtt{K} \begin{bmatrix} \mathtt{R} \mid \mathbf{t} \end{bmatrix} \mathbf{X}_w,
$$
where
\begin{equation}
[R | t] =
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_1\\
r_{21} & r_{22} & r_{23} & t_2\\
r_{31} & r_{32} & r_{33} & t_3
\end{bmatrix}
\end{equation}
If 
$\mathbf{X}_c =
\begin{bmatrix}
X_c\\
Y_c\\
Z_c
\end{bmatrix}$ and 
$\mathbf{X}_w =
\begin{bmatrix}
X_w\\
Y_w\\
Z_w\\
1,
\end{bmatrix}$
then
\begin{equation}
\begin{bmatrix}
X_c\\
Y_c\\
Z_c
\end{bmatrix} =
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_1\\
r_{21} & r_{22} & r_{23} & t_2\\
r_{31} & r_{32} & r_{33} & t_3
\end{bmatrix}
\begin{bmatrix}
X_w\\
Y_w\\
Z_w\\
1
\end{bmatrix}
\end{equation}


### Intrinsic camera parameters

These are the parameters that characterize the optical, geometric, and digital characteristics of the camera:
1. Perspective projection (focal length $f$).
2. The transformation between image plane coordinates and pixel coordinates.
3. The geometric distortion introduced by lens optics.

#### From Camera Coordinates to Image Plane Coordinates

We apply the following perspective projection:

$x = f\frac{X_c}{Z_c}$, $y = f\frac{X_c}{Z_c}$.

#### From Image Plane Coordinates to Pixel coordinates

<img src="img/lab04-18.png" width="600"/>

In matrix notation:
\begin{equation}
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix} \propto
\begin{bmatrix} x Z_c \\ y Z_c \\ Z_c \end{bmatrix} =
\begin{bmatrix}
\alpha_x & s & x_0\\
0 & \alpha_y & y_0\\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
X_c\\
Y_c\\
Z_c
\end{bmatrix}
\end{equation}

### From world coordinates to pixel coordinates

\begin{equation}
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix} \propto
\begin{bmatrix} x Z_c \\ y Z_c \\ Z_c \end{bmatrix} =
\begin{bmatrix}
\alpha_x & s & x_0\\
0 & \alpha_y & y_0\\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_1\\
r_{21} & r_{22} & r_{23} & t_2\\
r_{31} & r_{32} & r_{33} & t_3
\end{bmatrix}
\begin{bmatrix}
X_w\\
Y_w\\
Z_w\\
1
\end{bmatrix}
\end{equation}

Finally,
\begin{equation}
\begin{bmatrix}
x\\
y\\
1
\end{bmatrix} \propto
\begin{bmatrix}
m_{11} & m_{12} & m_{13} & m_{14}\\
m_{21} & m_{22} & m_{23} & m_{24}\\
m_{31} & m_{32} & m_{33} & m_{34}
\end{bmatrix}
\begin{bmatrix}
X_w\\
Y_w\\
Z_w\\
1
\end{bmatrix}
\end{equation}

### Image Distortion Due to Lens Optics

As the image input might get curved by the lens, we must rectify pixel coordinates before
attempting to calculate camera coordinate points using $\mathtt{K}^{-1}$.
We assume that $(x_d, y_d)$ is the pixel coordinate position of a point after distortion by the lens
before rectifying the image. The parameters used in OpenCV are
\begin{equation}
x = (x_d - x_c)(1+k_1r^2+k_2r^4 + k_3r^6)
\end{equation}
\begin{equation}
y = (y_d-y_c)(1+k_1r^2+k_2r^4 + k_3r^6)
\end{equation}

When $r^2 = (x_d - x_c)^2+(y_d - y_c)^2$, and $k_1$ and $k_2$ are intrinsic parameters.

The presence of the radial distortion manifests in form of the "barrel" or "fish-eye" effect.

Tangential distortion occurs because the image taking lenses are not perfectly parallel to the imaging plane. It can be represented via the formulas:

\begin{equation}
x = x_d+(2p_1(x_d - x_c)(y_d - x_c) + p_2(r^2+2(x_d-x_c)^2))
\end{equation}
\begin{equation}
y = y_d+(2p_1(r^2+2(y_d - y_c)^2)+2p_2(x_d - x_c)(y_d - y_c))
\end{equation}

finally,

\begin{equation}
x = x_d + (x_d - x_c)(1+k_1r^2+k_2r^4 + k_3r^6) +(2p_1(x_d - x_c)(y_d - x_c) + p_2(r^2+2(x_d-x_c)^2))
\end{equation}
\begin{equation}
y = y_d + (y_d-y_c)(1+k_1r^2+k_2r^4 + k_3r^6) +(2p_1(r^2+2(y_d - y_c)^2)+2p_2(x_d - x_c)(y_d - y_c))
\end{equation}

Thus we have 5 distortion parameters which in OpenCV are represented as a 5-element vector:

\begin{equation}\mathbf{C} = \begin{bmatrix}k1 & k2 & p1 & p2 & k3 \end{bmatrix}^\top\end{equation}

We can make C matrix by:

\begin{equation}
\begin{bmatrix} x \\ y \end{bmatrix} = \mathtt{A} \mathbf{C}'
\end{equation}

Where $\mathbf{C}'$ is
\begin{equation}\mathbf{C}' = \begin{bmatrix} k1 & k2 & p1 & p2 & k3 & 1\end{bmatrix}\end{equation}

and $\mathtt{A}$ is

\begin{equation}
\mathtt{A} =
\begin{bmatrix}
r^2(x_d-x_c) & r^4(x_d-x_c) & 2(x_d - x_c)(y_d - x_c) & (r^2+2(x_d-x_c)^2) & r^6(x_d-x_c) & (2x_d - x_c - x) \\
r^2(y_d-y_c) & r^4(y_d-y_c) & (r^2+2(y_d-y_c)^2) & 2(x_d - x_c)(y_d - x_c) & r^6(y_d-y_c) & (2y_d - y_c - y)
\end{bmatrix}
\end{equation}

## Camera Calibration

The calibration process is explained by a flowchart given below.

<img src="img/lab04-22.png" width="400"/>

### Chessboard calibration

#### 0. Firs of all, save and print the image (Chessboard) for use in camera.

<img src="img/lab04-19.png" width="600"/>

There are many patterns for calibration. However, openCV supports 3 patterns:
- Classical black-white chessboard (Image above)
- Symmetrical circle pattern

<img src="img/lab04-21.png" width="600"/>

- Asymmetrical circle pattern

#### 1. Define real world coordinates with checkerboard pattern

In the process of calibration we calculate the camera parameters by a set of know 3D points $(X_w, Y_w, Z_w)$ and their corresponding pixel location $(u,v)$ in the image.

For the 3D points we photograph a checkerboard pattern with known dimensions at many different orientations. The world coordinate is attached to the checkerboard and since all the corner points lie on a plane, we can arbitrarily choose Z_w for every point to be 0. Since points are equally spaced in the checkerboard, the $(X_w, Y_w)$ coordinates of each 3D point are easily defined by taking one point as reference $(0, 0)$ and defining remaining with respect to that reference point.

Write a program to get the image from camera or video.

If you use web camera, code example is at below:

### C++

In [ ]:
#include <opencv2/opencv.hpp>
#include <iostream>

using namespace cv;
using namespace std;
int main()
{
    int frameAdd = 0;
    Mat frame;
    int iKey = -1;
    //--- INITIALIZE VIDEOCAPTURE
    VideoCapture cap;
    // open the default camera using default API
    // cap.open(0);
    // OR advance usage: select any API backend
    int deviceID = 0;             // 0 = open default camera
    int apiID = cv::CAP_ANY;      // 0 = autodetect default API
    // open selected camera using selected API
    cap.open(deviceID, apiID);
    // check if we succeeded
    if (!cap.isOpened()) {
        cerr << "ERROR! Unable to open camera\n";
        return -1;
    }
    //--- GRAB AND WRITE LOOP
    cout << "Start grabbing" << endl
        << "Press s to save images and q to terminate" << endl;
    for (;;)
    {
        // wait for a new frame from camera and store it into 'frame'
        cap.read(frame);
        // check if we succeeded
        if (frame.empty()) {
            cerr << "ERROR! blank frame grabbed\n";
            break;
        }
        // show live and wait for a key with timeout long enough to show images
        imshow("Live", frame);
        iKey = waitKey(5);
        if (iKey == 's' || iKey == 'S')
        {
            imwrite("./images/frame" + to_string(frameAdd) + ".jpg", frame);
            wantFrame[frameAdd] = frame.clone();
            frameAdd++;
            count << "Frame: " << frameAdd << " has been saved." << endl;
        }
        else if (iKey == 'q' || iKey == 'Q')
        {
            break;
        }
    }
    // the camera will be deinitialized automatically in VideoCapture destructor
    return 0;
}

### Python

In [ ]:
import cv2
import numpy as np
import os
import glob

cap = cv2.VideoCapture()
cap.open(0, cv2.CAP_ANY);
if not cap.isOpened():
    print("ERROR! Unable to open camera\n")
    exit()
print("Start grabbing")
print("Press s to save images and q to terminate")
frameAdd = 0
while True:
    _, frame = cap.read()
    if frame is None:
        print("ERROR! blank frame grabbed\n")
        exit()
    cv2.imshow("Live", frame)
    iKey = cv2.waitKey(5)
    if iKey == ord('s') or iKey == ord('S'):
        cv2.imwrite("./images/frame" + str(frameAdd) + ".jpg", frame)
        frameAdd += 1
        print("Frame: ", frameAdd, " has been saved.")
    elif iKey == ord('q') or iKey == ord('Q'):
        break

Run the web camera, you will see the camera with chessboard as below:

<img src="img/lab04-20.png" width="600"/>

####  2. Take several pictures for the checkerboard (generally more than 10)

Save and put them to <code>./images</code>

<img src="img/lab04-23.gif" width="300"/>

There are two cases here

    (1) To calibrate the distortion coefficient and camera internal parameters, the photographs need to contain a complete
    chessboard, and at the same time require different distances, different orientations, and different tilt angles of the
    chess board.

    (2) Calibration distortion coefficient, camera internal parameters and camera external parameters. The picture contains
    the above requirements. At the same time, each photo in the calibration program generated results will calculate an
    external camera parameter. Therefore, according to actual needs, add a few photos of the chessboard at the working
    position. (It is recommended to use the solvePnP function to obtain external camera parameters)

#### 3. 1. Find checkerboard corners

OpenCV provides a builtin function called findChessboardCorners that looks for a checkerboard and returns the coordinates of the corners. Let’ see the usage in the code block below. Its usage is given by

### C++

In [ ]:
bool findChessboardCorners(InputArray image, 
                           Size patternSize, 
                           OutputArray corners, 
                           int flags=CALIB_CB_ADAPTIVE_THRESH+CALIB_CB_NORMALIZE_IMAGE );

### Python

In [ ]:
retval, corners = cv2.findChessboardCorners(image, patternSize, flags)

| Variable | Meaning |
| :--- | :--- |
| **image** | Source chessboard view. It must be an 8-bit grayscale or color image. |
| **patternSize** | Number of inner corners per a chessboard row and column ( patternSize = cvSize (points_per_row, points_per_colum) = cvSize(columns,rows) ). |
| **corners** | Output array of detected corners. |
| **flags** | Various operation flags. You have to worry about these only when things do not work well. Go with the default. |

####  3. 2. Refine checkerboard corners

Good calibration is all about precision. To get good results it is important to obtain the location of corners with sub-pixel level of accuracy.

OpenCV’s function cornerSubPix takes in the original image, and the location of corners, and looks for the best corner location inside a small neighborhood of the original location. The algorithm is iterative in nature and therefore we need to specify the termination criteria ( e.g. number of iterations and/or the accuracy )

### C++

In [ ]:
bool findChessboardCorners(InputArray image, 
                           Size patternSize,
                           OutputArray corners,
                           int flags = CALIB_CB_ADAPTIVE_THRESH + CALIB_CB_NORMALIZE_IMAGE );

### Python

In [ ]:
retval, corners = cv2.findChessboardCorners(image, patternSize, flags)

| Variable | Meaning |
| :--- | :--- |
| **image** | Input image. |
| **corners** | Initial coordinates of the input corners and refined coordinates provided for output. |
| **winSize** | Half of the side length of the search window. |
| **zeroZone** | Half of the size of the dead region in the middle of the search zone over which the summation in the formula below is not done. It is used sometimes to avoid possible singularities of the autocorrelation matrix. The value of (-1,-1) indicates that there is no such a size. |
|**criteria** | Criteria for termination of the iterative process of corner refinement. That is, the process of corner position refinement stops either after criteria.maxCount iterations or when the corner position moves by less than criteria.epsilon on some iteration.|

#### 4. Calibrate Camera

The final step of calibration is to pass the 3D points in world coordinates and their 2D locations in all images to OpenCV’s calibrateCamera method. The implementation is based on a paper by Zhengyou Zhang. The math is a bit involved and requires a background in linear algebra.

Let’s look at the syntax for calibrateCamera

### C++

In [ ]:
double calibrateCamera(InputArrayOfArrays objectPoints,
                       InputArrayOfArrays imagePoints,
                       Size imageSize,
                       InputOutputArray cameraMatrix,
                       InputOutputArray distCoeffs,
                       OutputArrayOfArrays rvecs,
                       OutputArrayOfArrays tvecs);

### Python

In [ ]:
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, imageSize)

| Variable | Meaning |
| :--- | :--- |
| **objectPoints** | A vector of vector of 3D points. The outer vector contains as many elements as the number of the pattern views. |
| **imagePoints** | A vector of vectors of the 2D image points. |
| **imageSize** | Size of the image |
| **cameraMatrix** | Intrinsic camera matrix |
|**distCoeffs** | Lens distortion coefficients. These coefficients will be explained in a future post.|
| **rvecs** | Rotation specified as a 3×1 vector. The direction of the vector specifies the axis of rotation and the magnitude of the vector specifies the angle of rotation. |
| **tvecs** | 3×1 Translation vector. |

### Full code of camera calibration

### C++

In [ ]:
#include <opencv2/opencv.hpp>
#include <opencv2/calib3d/calib3d.hpp>
#include <opencv2/highgui/highgui.hpp>
#include <opencv2/imgproc/imgproc.hpp>
#include <stdio.h>
#include <iostream>

// Defining the dimensions of checkerboard
int CHECKERBOARD[2]{8,11}; // width, height

int main()
{
  // Creating vector to store vectors of 3D points for each checkerboard image
  std::vector<std::vector<cv::Point3f> > objpoints;

  // Creating vector to store vectors of 2D points for each checkerboard image
  std::vector<std::vector<cv::Point2f> > imgpoints;

  // Defining the world coordinates for 3D points
  std::vector<cv::Point3f> objp;
  for(int i = 0; i<CHECKERBOARD[1]; i++)
  {
    for(int j = 0; j<CHECKERBOARD[0]; j++)
      objp.push_back(cv::Point3f(j,i,0));
  }


  // Extracting path of individual image stored in a given directory
  std::vector<cv::String> images;
  // Path of the folder containing checkerboard images
  std::string path = "./images/*.jpg";

  cv::glob(path, images);

  cv::Mat frame, gray;
  // vector to store the pixel coordinates of detected checker board corners 
  std::vector<cv::Point2f> corner_pts;
  bool success;

  // Looping over all the images in the directory
  for(int i{0}; i<images.size(); i++)
  {
    frame = cv::imread(images[i]);
    cv::cvtColor(frame,gray,cv::COLOR_BGR2GRAY);

    // Finding checker board corners
    // If desired number of corners are found in the image then success = true  
    success = cv::findChessboardCorners(gray, cv::Size(CHECKERBOARD[0], CHECKERBOARD[1]), corner_pts, CV_CALIB_CB_ADAPTIVE_THRESH | CV_CALIB_CB_FAST_CHECK | CV_CALIB_CB_NORMALIZE_IMAGE);
    
    /* 
     * If desired number of corner are detected,
     * we refine the pixel coordinates and display 
     * them on the images of checker board
    */
    if(success)
    {
      cv::TermCriteria criteria(CV_TERMCRIT_EPS | CV_TERMCRIT_ITER, 30, 0.001);
      
      // refining pixel coordinates for given 2d points.
      cv::cornerSubPix(gray,corner_pts,cv::Size(11,11), cv::Size(-1,-1),criteria);
      
      // Displaying the detected corner points on the checker board
      cv::drawChessboardCorners(frame, cv::Size(CHECKERBOARD[0], CHECKERBOARD[1]), corner_pts, success);
      
      objpoints.push_back(objp);
      imgpoints.push_back(corner_pts);
    }

    cv::imshow("Image",frame);
    cv::waitKey(0);
  }

  cv::destroyAllWindows();

  cv::Mat cameraMatrix,distCoeffs,R,T;

  /*
   * Performing camera calibration by 
   * passing the value of known 3D points (objpoints)
   * and corresponding pixel coordinates of the 
   * detected corners (imgpoints)
  */
  cv::calibrateCamera(objpoints, imgpoints, cv::Size(gray.rows,gray.cols), cameraMatrix, distCoeffs, R, T);

  std::cout << "cameraMatrix : " << cameraMatrix << std::endl;
  std::cout << "distCoeffs : " << distCoeffs << std::endl;
  std::cout << "Rotation vector : " << R << std::endl;
  std::cout << "Translation vector : " << T << std::endl;

  return 0;
}

### Python

In [ ]:
#!/usr/bin/env python

import cv2
import numpy as np
import os
import glob

# Defining the dimensions of checkerboard
CHECKERBOARD = (8,11)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 


# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

# Extracting path of individual image stored in a given directory
images = glob.glob('./images/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    
    cv2.imshow('img',img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

h,w = img.shape[:2]

"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

# Show images of undistorted
for fname in images:
    img = cv2.imread(fname)
    res = cv2.undistort(img, mtx, dist)
    cv2.imshow('img',img)
    cv2.imshow('res',res)
    cv2.waitKey(0)

### Result of the images

You can see the result like this.

Source image
<img src="img/lab04-26.png" width="600"/>

Chessboard detection
<img src="img/lab04-24.png" width="600"/>

Rectified image
<img src="img/lab04-25.png" width="600"/>

### More fancier in undistorted

After undistorted, the result of rectified image may need to be crops.

### Python

In [ ]:
# Way 1: This is the easiest way. Just call the function and use ROI obtained above to crop the result.
for fname in images:
    img = cv2.imread(fname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    
    res = cv2.undistort(img, mtx, dist)
    
    # crop the image
    x, y, w, h = roi
    res = res[y:y+h, x:x+w]
    
    cv2.imshow('img',img)
    cv2.imshow('res',res)
    cv2.waitKey(0)

In [ ]:
# Way 2: This way is a little bit more difficult. 
# First, find a mapping function from the distorted image to the undistorted image.
# Then use the remap function.

for fname in images:
    img = cv2.imread(fname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
    res = cv2.remap(img, mapx, mapy, cv.INTER_LINEAR)
    
    # crop the image
    x, y, w, h = roi
    res = res[y:y+h, x:x+w]
    
    cv2.imshow('img',img)
    cv2.imshow('res',res)
    cv2.waitKey(0)

### Re-projection Error

Re-projection error gives a good estimation of just how exact the found parameters are. The closer the re-projection error is to zero, the more accurate the parameters we found are. Given the intrinsic, distortion, rotation and translation matrices, we must first transform the object point to image point using cv.projectPoints(). Then, we can calculate the absolute norm between what we got with our transformation and the corner finding algorithm. To find the average error, we calculate the arithmetical mean of the errors calculated for all the calibration images.

### Python

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

### Save calibration file

As the result, save the calibration file into yml file.

### Full code of Calibration from scratch

Please look at https://github.com/opencv/opencv/blob/master/samples/cpp/tutorial_code/calib3d/camera_calibration/camera_calibration.cpp
There is the source code in C++ from scratch. It may useful when you need to write it as commercial product.

### C++

In [ ]:
#include <iostream>
#include <sstream>
#include <string>
#include <ctime>
#include <cstdio>

#include <opencv2/core.hpp>
#include <opencv2/core/utility.hpp>
#include <opencv2/imgproc.hpp>
#include <opencv2/calib3d.hpp>
#include <opencv2/imgcodecs.hpp>
#include <opencv2/videoio.hpp>
#include <opencv2/highgui.hpp>

using namespace cv;
using namespace std;

class Settings
{
public:
    Settings() : goodInput(false) {}
    enum Pattern { NOT_EXISTING, CHESSBOARD, CIRCLES_GRID, ASYMMETRIC_CIRCLES_GRID };
    enum InputType { INVALID, CAMERA, VIDEO_FILE, IMAGE_LIST };

    void write(FileStorage& fs) const                        //Write serialization for this class
    {
        fs << "{"
                  << "BoardSize_Width"  << boardSize.width
                  << "BoardSize_Height" << boardSize.height
                  << "Square_Size"         << squareSize
                  << "Calibrate_Pattern" << patternToUse
                  << "Calibrate_NrOfFrameToUse" << nrFrames
                  << "Calibrate_FixAspectRatio" << aspectRatio
                  << "Calibrate_AssumeZeroTangentialDistortion" << calibZeroTangentDist
                  << "Calibrate_FixPrincipalPointAtTheCenter" << calibFixPrincipalPoint

                  << "Write_DetectedFeaturePoints" << writePoints
                  << "Write_extrinsicParameters"   << writeExtrinsics
                  << "Write_gridPoints" << writeGrid
                  << "Write_outputFileName"  << outputFileName

                  << "Show_UndistortedImage" << showUndistorted

                  << "Input_FlipAroundHorizontalAxis" << flipVertical
                  << "Input_Delay" << delay
                  << "Input" << input
           << "}";
    }
    void read(const FileNode& node)                          //Read serialization for this class
    {
        node["BoardSize_Width" ] >> boardSize.width;
        node["BoardSize_Height"] >> boardSize.height;
        node["Calibrate_Pattern"] >> patternToUse;
        node["Square_Size"]  >> squareSize;
        node["Calibrate_NrOfFrameToUse"] >> nrFrames;
        node["Calibrate_FixAspectRatio"] >> aspectRatio;
        node["Write_DetectedFeaturePoints"] >> writePoints;
        node["Write_extrinsicParameters"] >> writeExtrinsics;
        node["Write_gridPoints"] >> writeGrid;
        node["Write_outputFileName"] >> outputFileName;
        node["Calibrate_AssumeZeroTangentialDistortion"] >> calibZeroTangentDist;
        node["Calibrate_FixPrincipalPointAtTheCenter"] >> calibFixPrincipalPoint;
        node["Calibrate_UseFisheyeModel"] >> useFisheye;
        node["Input_FlipAroundHorizontalAxis"] >> flipVertical;
        node["Show_UndistortedImage"] >> showUndistorted;
        node["Input"] >> input;
        node["Input_Delay"] >> delay;
        node["Fix_K1"] >> fixK1;
        node["Fix_K2"] >> fixK2;
        node["Fix_K3"] >> fixK3;
        node["Fix_K4"] >> fixK4;
        node["Fix_K5"] >> fixK5;

        validate();
    }
    void validate()
    {
        goodInput = true;
        if (boardSize.width <= 0 || boardSize.height <= 0)
        {
            cerr << "Invalid Board size: " << boardSize.width << " " << boardSize.height << endl;
            goodInput = false;
        }
        if (squareSize <= 10e-6)
        {
            cerr << "Invalid square size " << squareSize << endl;
            goodInput = false;
        }
        if (nrFrames <= 0)
        {
            cerr << "Invalid number of frames " << nrFrames << endl;
            goodInput = false;
        }

        if (input.empty())      // Check for valid input
                inputType = INVALID;
        else
        {
            if (input[0] >= '0' && input[0] <= '9')
            {
                stringstream ss(input);
                ss >> cameraID;
                inputType = CAMERA;
            }
            else
            {
                if (isListOfImages(input) && readStringList(input, imageList))
                {
                    inputType = IMAGE_LIST;
                    nrFrames = (nrFrames < (int)imageList.size()) ? nrFrames : (int)imageList.size();
                }
                else
                    inputType = VIDEO_FILE;
            }
            if (inputType == CAMERA)
                inputCapture.open(cameraID);
            if (inputType == VIDEO_FILE)
                inputCapture.open(input);
            if (inputType != IMAGE_LIST && !inputCapture.isOpened())
                    inputType = INVALID;
        }
        if (inputType == INVALID)
        {
            cerr << " Input does not exist: " << input;
            goodInput = false;
        }

        flag = 0;
        if(calibFixPrincipalPoint) flag |= CALIB_FIX_PRINCIPAL_POINT;
        if(calibZeroTangentDist)   flag |= CALIB_ZERO_TANGENT_DIST;
        if(aspectRatio)            flag |= CALIB_FIX_ASPECT_RATIO;
        if(fixK1)                  flag |= CALIB_FIX_K1;
        if(fixK2)                  flag |= CALIB_FIX_K2;
        if(fixK3)                  flag |= CALIB_FIX_K3;
        if(fixK4)                  flag |= CALIB_FIX_K4;
        if(fixK5)                  flag |= CALIB_FIX_K5;

        if (useFisheye) {
            // the fisheye model has its own enum, so overwrite the flags
            flag = fisheye::CALIB_FIX_SKEW | fisheye::CALIB_RECOMPUTE_EXTRINSIC;
            if(fixK1)                   flag |= fisheye::CALIB_FIX_K1;
            if(fixK2)                   flag |= fisheye::CALIB_FIX_K2;
            if(fixK3)                   flag |= fisheye::CALIB_FIX_K3;
            if(fixK4)                   flag |= fisheye::CALIB_FIX_K4;
            if (calibFixPrincipalPoint) flag |= fisheye::CALIB_FIX_PRINCIPAL_POINT;
        }

        calibrationPattern = NOT_EXISTING;
        if (!patternToUse.compare("CHESSBOARD")) calibrationPattern = CHESSBOARD;
        if (!patternToUse.compare("CIRCLES_GRID")) calibrationPattern = CIRCLES_GRID;
        if (!patternToUse.compare("ASYMMETRIC_CIRCLES_GRID")) calibrationPattern = ASYMMETRIC_CIRCLES_GRID;
        if (calibrationPattern == NOT_EXISTING)
        {
            cerr << " Camera calibration mode does not exist: " << patternToUse << endl;
            goodInput = false;
        }
        atImageList = 0;

    }
    Mat nextImage()
    {
        Mat result;
        if( inputCapture.isOpened() )
        {
            Mat view0;
            inputCapture >> view0;
            view0.copyTo(result);
        }
        else if( atImageList < imageList.size() )
            result = imread(imageList[atImageList++], IMREAD_COLOR);

        return result;
    }

    static bool readStringList( const string& filename, vector<string>& l )
    {
        l.clear();
        FileStorage fs(filename, FileStorage::READ);
        if( !fs.isOpened() )
            return false;
        FileNode n = fs.getFirstTopLevelNode();
        if( n.type() != FileNode::SEQ )
            return false;
        FileNodeIterator it = n.begin(), it_end = n.end();
        for( ; it != it_end; ++it )
            l.push_back((string)*it);
        return true;
    }

    static bool isListOfImages( const string& filename)
    {
        string s(filename);
        // Look for file extension
        if( s.find(".xml") == string::npos && s.find(".yaml") == string::npos && s.find(".yml") == string::npos )
            return false;
        else
            return true;
    }
public:
    Size boardSize;              // The size of the board -> Number of items by width and height
    Pattern calibrationPattern;  // One of the Chessboard, circles, or asymmetric circle pattern
    float squareSize;            // The size of a square in your defined unit (point, millimeter,etc).
    int nrFrames;                // The number of frames to use from the input for calibration
    float aspectRatio;           // The aspect ratio
    int delay;                   // In case of a video input
    bool writePoints;            // Write detected feature points
    bool writeExtrinsics;        // Write extrinsic parameters
    bool writeGrid;              // Write refined 3D target grid points
    bool calibZeroTangentDist;   // Assume zero tangential distortion
    bool calibFixPrincipalPoint; // Fix the principal point at the center
    bool flipVertical;           // Flip the captured images around the horizontal axis
    string outputFileName;       // The name of the file where to write
    bool showUndistorted;        // Show undistorted images after calibration
    string input;                // The input ->
    bool useFisheye;             // use fisheye camera model for calibration
    bool fixK1;                  // fix K1 distortion coefficient
    bool fixK2;                  // fix K2 distortion coefficient
    bool fixK3;                  // fix K3 distortion coefficient
    bool fixK4;                  // fix K4 distortion coefficient
    bool fixK5;                  // fix K5 distortion coefficient

    int cameraID;
    vector<string> imageList;
    size_t atImageList;
    VideoCapture inputCapture;
    InputType inputType;
    bool goodInput;
    int flag;

private:
    string patternToUse;


};

static inline void read(const FileNode& node, Settings& x, const Settings& default_value = Settings())
{
    if(node.empty())
        x = default_value;
    else
        x.read(node);
}

enum { DETECTION = 0, CAPTURING = 1, CALIBRATED = 2 };

bool runCalibrationAndSave(Settings& s, Size imageSize, Mat&  cameraMatrix, Mat& distCoeffs,
                           vector<vector<Point2f> > imagePoints, float grid_width, bool release_object);

int main(int argc, char* argv[])
{
    const String keys
        = "{help h usage ? |           | print this message            }"
          "{@settings      |default.xml| input setting file            }"
          "{d              |           | actual distance between top-left and top-right corners of "
          "the calibration grid }"
          "{winSize        | 11        | Half of search window for cornerSubPix }";
    CommandLineParser parser(argc, argv, keys);
    parser.about("This is a camera calibration sample.\n"
                 "Usage: camera_calibration [configuration_file -- default ./default.xml]\n"
                 "Near the sample file you'll find the configuration file, which has detailed help of "
                 "how to edit it. It may be any OpenCV supported file format XML/YAML.");
    if (!parser.check()) {
        parser.printErrors();
        return 0;
    }

    if (parser.has("help")) {
        parser.printMessage();
        return 0;
    }

    //! [file_read]
    Settings s;
    const string inputSettingsFile = parser.get<string>(0);
    FileStorage fs(inputSettingsFile, FileStorage::READ); // Read the settings
    if (!fs.isOpened())
    {
        cout << "Could not open the configuration file: \"" << inputSettingsFile << "\"" << endl;
        parser.printMessage();
        return -1;
    }
    fs["Settings"] >> s;
    fs.release();                                         // close Settings file
    //! [file_read]

    //FileStorage fout("settings.yml", FileStorage::WRITE); // write config as YAML
    //fout << "Settings" << s;

    if (!s.goodInput)
    {
        cout << "Invalid input detected. Application stopping. " << endl;
        return -1;
    }

    int winSize = parser.get<int>("winSize");

    float grid_width = s.squareSize * (s.boardSize.width - 1);
    bool release_object = false;
    if (parser.has("d")) {
        grid_width = parser.get<float>("d");
        release_object = true;
    }

    vector<vector<Point2f> > imagePoints;
    Mat cameraMatrix, distCoeffs;
    Size imageSize;
    int mode = s.inputType == Settings::IMAGE_LIST ? CAPTURING : DETECTION;
    clock_t prevTimestamp = 0;
    const Scalar RED(0,0,255), GREEN(0,255,0);
    const char ESC_KEY = 27;

    //! [get_input]
    for(;;)
    {
        Mat view;
        bool blinkOutput = false;

        view = s.nextImage();

        //-----  If no more image, or got enough, then stop calibration and show result -------------
        if( mode == CAPTURING && imagePoints.size() >= (size_t)s.nrFrames )
        {
          if(runCalibrationAndSave(s, imageSize,  cameraMatrix, distCoeffs, imagePoints, grid_width,
                                   release_object))
              mode = CALIBRATED;
          else
              mode = DETECTION;
        }
        if(view.empty())          // If there are no more images stop the loop
        {
            // if calibration threshold was not reached yet, calibrate now
            if( mode != CALIBRATED && !imagePoints.empty() )
                runCalibrationAndSave(s, imageSize,  cameraMatrix, distCoeffs, imagePoints, grid_width,
                                      release_object);
            break;
        }
        //! [get_input]

        imageSize = view.size();  // Format input image.
        if( s.flipVertical )    flip( view, view, 0 );

        //! [find_pattern]
        vector<Point2f> pointBuf;

        bool found;

        int chessBoardFlags = CALIB_CB_ADAPTIVE_THRESH | CALIB_CB_NORMALIZE_IMAGE;

        if(!s.useFisheye) {
            // fast check erroneously fails with high distortions like fisheye
            chessBoardFlags |= CALIB_CB_FAST_CHECK;
        }

        switch( s.calibrationPattern ) // Find feature points on the input format
        {
        case Settings::CHESSBOARD:
            found = findChessboardCorners( view, s.boardSize, pointBuf, chessBoardFlags);
            break;
        case Settings::CIRCLES_GRID:
            found = findCirclesGrid( view, s.boardSize, pointBuf );
            break;
        case Settings::ASYMMETRIC_CIRCLES_GRID:
            found = findCirclesGrid( view, s.boardSize, pointBuf, CALIB_CB_ASYMMETRIC_GRID );
            break;
        default:
            found = false;
            break;
        }
        //! [find_pattern]
        //! [pattern_found]
        if ( found)                // If done with success,
        {
              // improve the found corners' coordinate accuracy for chessboard
                if( s.calibrationPattern == Settings::CHESSBOARD)
                {
                    Mat viewGray;
                    cvtColor(view, viewGray, COLOR_BGR2GRAY);
                    cornerSubPix( viewGray, pointBuf, Size(winSize,winSize),
                        Size(-1,-1), TermCriteria( TermCriteria::EPS+TermCriteria::COUNT, 30, 0.0001 ));
                }

                if( mode == CAPTURING &&  // For camera only take new samples after delay time
                    (!s.inputCapture.isOpened() || clock() - prevTimestamp > s.delay*1e-3*CLOCKS_PER_SEC) )
                {
                    imagePoints.push_back(pointBuf);
                    prevTimestamp = clock();
                    blinkOutput = s.inputCapture.isOpened();
                }

                // Draw the corners.
                drawChessboardCorners( view, s.boardSize, Mat(pointBuf), found );
        }
        //! [pattern_found]
        //----------------------------- Output Text ------------------------------------------------
        //! [output_text]
        string msg = (mode == CAPTURING) ? "100/100" :
                      mode == CALIBRATED ? "Calibrated" : "Press 'g' to start";
        int baseLine = 0;
        Size textSize = getTextSize(msg, 1, 1, 1, &baseLine);
        Point textOrigin(view.cols - 2*textSize.width - 10, view.rows - 2*baseLine - 10);

        if( mode == CAPTURING )
        {
            if(s.showUndistorted)
                msg = cv::format( "%d/%d Undist", (int)imagePoints.size(), s.nrFrames );
            else
                msg = cv::format( "%d/%d", (int)imagePoints.size(), s.nrFrames );
        }

        putText( view, msg, textOrigin, 1, 1, mode == CALIBRATED ?  GREEN : RED);

        if( blinkOutput )
            bitwise_not(view, view);
        //! [output_text]
        //------------------------- Video capture  output  undistorted ------------------------------
        //! [output_undistorted]
        if( mode == CALIBRATED && s.showUndistorted )
        {
            Mat temp = view.clone();
            if (s.useFisheye)
            {
                Mat newCamMat;
                fisheye::estimateNewCameraMatrixForUndistortRectify(cameraMatrix, distCoeffs, imageSize,
                                                                    Matx33d::eye(), newCamMat, 1);
                cv::fisheye::undistortImage(temp, view, cameraMatrix, distCoeffs, newCamMat);
            }
            else
              undistort(temp, view, cameraMatrix, distCoeffs);
        }
        //! [output_undistorted]
        //------------------------------ Show image and check for input commands -------------------
        //! [await_input]
        imshow("Image View", view);
        char key = (char)waitKey(s.inputCapture.isOpened() ? 50 : s.delay);

        if( key  == ESC_KEY )
            break;

        if( key == 'u' && mode == CALIBRATED )
           s.showUndistorted = !s.showUndistorted;

        if( s.inputCapture.isOpened() && key == 'g' )
        {
            mode = CAPTURING;
            imagePoints.clear();
        }
        //! [await_input]
    }

    // -----------------------Show the undistorted image for the image list ------------------------
    //! [show_results]
    if( s.inputType == Settings::IMAGE_LIST && s.showUndistorted && !cameraMatrix.empty())
    {
        Mat view, rview, map1, map2;

        if (s.useFisheye)
        {
            Mat newCamMat;
            fisheye::estimateNewCameraMatrixForUndistortRectify(cameraMatrix, distCoeffs, imageSize,
                                                                Matx33d::eye(), newCamMat, 1);
            fisheye::initUndistortRectifyMap(cameraMatrix, distCoeffs, Matx33d::eye(), newCamMat, imageSize,
                                             CV_16SC2, map1, map2);
        }
        else
        {
            initUndistortRectifyMap(
                cameraMatrix, distCoeffs, Mat(),
                getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, imageSize, 1, imageSize, 0), imageSize,
                CV_16SC2, map1, map2);
        }

        for(size_t i = 0; i < s.imageList.size(); i++ )
        {
            view = imread(s.imageList[i], IMREAD_COLOR);
            if(view.empty())
                continue;
            remap(view, rview, map1, map2, INTER_LINEAR);
            imshow("Image View", rview);
            char c = (char)waitKey();
            if( c  == ESC_KEY || c == 'q' || c == 'Q' )
                break;
        }
    }
    //! [show_results]

    return 0;
}

//! [compute_errors]
static double computeReprojectionErrors( const vector<vector<Point3f> >& objectPoints,
                                         const vector<vector<Point2f> >& imagePoints,
                                         const vector<Mat>& rvecs, const vector<Mat>& tvecs,
                                         const Mat& cameraMatrix , const Mat& distCoeffs,
                                         vector<float>& perViewErrors, bool fisheye)
{
    vector<Point2f> imagePoints2;
    size_t totalPoints = 0;
    double totalErr = 0, err;
    perViewErrors.resize(objectPoints.size());

    for(size_t i = 0; i < objectPoints.size(); ++i )
    {
        if (fisheye)
        {
            fisheye::projectPoints(objectPoints[i], imagePoints2, rvecs[i], tvecs[i], cameraMatrix,
                                   distCoeffs);
        }
        else
        {
            projectPoints(objectPoints[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs, imagePoints2);
        }
        err = norm(imagePoints[i], imagePoints2, NORM_L2);

        size_t n = objectPoints[i].size();
        perViewErrors[i] = (float) std::sqrt(err*err/n);
        totalErr        += err*err;
        totalPoints     += n;
    }

    return std::sqrt(totalErr/totalPoints);
}
//! [compute_errors]
//! [board_corners]
static void calcBoardCornerPositions(Size boardSize, float squareSize, vector<Point3f>& corners,
                                     Settings::Pattern patternType /*= Settings::CHESSBOARD*/)
{
    corners.clear();

    switch(patternType)
    {
    case Settings::CHESSBOARD:
    case Settings::CIRCLES_GRID:
        for( int i = 0; i < boardSize.height; ++i )
            for( int j = 0; j < boardSize.width; ++j )
                corners.push_back(Point3f(j*squareSize, i*squareSize, 0));
        break;

    case Settings::ASYMMETRIC_CIRCLES_GRID:
        for( int i = 0; i < boardSize.height; i++ )
            for( int j = 0; j < boardSize.width; j++ )
                corners.push_back(Point3f((2*j + i % 2)*squareSize, i*squareSize, 0));
        break;
    default:
        break;
    }
}
//! [board_corners]
static bool runCalibration( Settings& s, Size& imageSize, Mat& cameraMatrix, Mat& distCoeffs,
                            vector<vector<Point2f> > imagePoints, vector<Mat>& rvecs, vector<Mat>& tvecs,
                            vector<float>& reprojErrs,  double& totalAvgErr, vector<Point3f>& newObjPoints,
                            float grid_width, bool release_object)
{
    //! [fixed_aspect]
    cameraMatrix = Mat::eye(3, 3, CV_64F);
    if( !s.useFisheye && s.flag & CALIB_FIX_ASPECT_RATIO )
        cameraMatrix.at<double>(0,0) = s.aspectRatio;
    //! [fixed_aspect]
    if (s.useFisheye) {
        distCoeffs = Mat::zeros(4, 1, CV_64F);
    } else {
        distCoeffs = Mat::zeros(8, 1, CV_64F);
    }

    vector<vector<Point3f> > objectPoints(1);
    calcBoardCornerPositions(s.boardSize, s.squareSize, objectPoints[0], s.calibrationPattern);
    objectPoints[0][s.boardSize.width - 1].x = objectPoints[0][0].x + grid_width;
    newObjPoints = objectPoints[0];

    objectPoints.resize(imagePoints.size(),objectPoints[0]);

    //Find intrinsic and extrinsic camera parameters
    double rms;

    if (s.useFisheye) {
        Mat _rvecs, _tvecs;
        rms = fisheye::calibrate(objectPoints, imagePoints, imageSize, cameraMatrix, distCoeffs, _rvecs,
                                 _tvecs, s.flag);

        rvecs.reserve(_rvecs.rows);
        tvecs.reserve(_tvecs.rows);
        for(int i = 0; i < int(objectPoints.size()); i++){
            rvecs.push_back(_rvecs.row(i));
            tvecs.push_back(_tvecs.row(i));
        }
    } else {
        int iFixedPoint = -1;
        if (release_object)
            iFixedPoint = s.boardSize.width - 1;
        rms = calibrateCameraRO(objectPoints, imagePoints, imageSize, iFixedPoint,
                                cameraMatrix, distCoeffs, rvecs, tvecs, newObjPoints,
                                s.flag | CALIB_USE_LU);
    }

    if (release_object) {
        cout << "New board corners: " << endl;
        cout << newObjPoints[0] << endl;
        cout << newObjPoints[s.boardSize.width - 1] << endl;
        cout << newObjPoints[s.boardSize.width * (s.boardSize.height - 1)] << endl;
        cout << newObjPoints.back() << endl;
    }

    cout << "Re-projection error reported by calibrateCamera: "<< rms << endl;

    bool ok = checkRange(cameraMatrix) && checkRange(distCoeffs);

    objectPoints.clear();
    objectPoints.resize(imagePoints.size(), newObjPoints);
    totalAvgErr = computeReprojectionErrors(objectPoints, imagePoints, rvecs, tvecs, cameraMatrix,
                                            distCoeffs, reprojErrs, s.useFisheye);

    return ok;
}

// Print camera parameters to the output file
static void saveCameraParams( Settings& s, Size& imageSize, Mat& cameraMatrix, Mat& distCoeffs,
                              const vector<Mat>& rvecs, const vector<Mat>& tvecs,
                              const vector<float>& reprojErrs, const vector<vector<Point2f> >& imagePoints,
                              double totalAvgErr, const vector<Point3f>& newObjPoints )
{
    FileStorage fs( s.outputFileName, FileStorage::WRITE );

    time_t tm;
    time( &tm );
    struct tm *t2 = localtime( &tm );
    char buf[1024];
    strftime( buf, sizeof(buf), "%c", t2 );

    fs << "calibration_time" << buf;

    if( !rvecs.empty() || !reprojErrs.empty() )
        fs << "nr_of_frames" << (int)std::max(rvecs.size(), reprojErrs.size());
    fs << "image_width" << imageSize.width;
    fs << "image_height" << imageSize.height;
    fs << "board_width" << s.boardSize.width;
    fs << "board_height" << s.boardSize.height;
    fs << "square_size" << s.squareSize;

    if( !s.useFisheye && s.flag & CALIB_FIX_ASPECT_RATIO )
        fs << "fix_aspect_ratio" << s.aspectRatio;

    if (s.flag)
    {
        std::stringstream flagsStringStream;
        if (s.useFisheye)
        {
            flagsStringStream << "flags:"
                << (s.flag & fisheye::CALIB_FIX_SKEW ? " +fix_skew" : "")
                << (s.flag & fisheye::CALIB_FIX_K1 ? " +fix_k1" : "")
                << (s.flag & fisheye::CALIB_FIX_K2 ? " +fix_k2" : "")
                << (s.flag & fisheye::CALIB_FIX_K3 ? " +fix_k3" : "")
                << (s.flag & fisheye::CALIB_FIX_K4 ? " +fix_k4" : "")
                << (s.flag & fisheye::CALIB_RECOMPUTE_EXTRINSIC ? " +recompute_extrinsic" : "");
        }
        else
        {
            flagsStringStream << "flags:"
                << (s.flag & CALIB_USE_INTRINSIC_GUESS ? " +use_intrinsic_guess" : "")
                << (s.flag & CALIB_FIX_ASPECT_RATIO ? " +fix_aspectRatio" : "")
                << (s.flag & CALIB_FIX_PRINCIPAL_POINT ? " +fix_principal_point" : "")
                << (s.flag & CALIB_ZERO_TANGENT_DIST ? " +zero_tangent_dist" : "")
                << (s.flag & CALIB_FIX_K1 ? " +fix_k1" : "")
                << (s.flag & CALIB_FIX_K2 ? " +fix_k2" : "")
                << (s.flag & CALIB_FIX_K3 ? " +fix_k3" : "")
                << (s.flag & CALIB_FIX_K4 ? " +fix_k4" : "")
                << (s.flag & CALIB_FIX_K5 ? " +fix_k5" : "");
        }
        fs.writeComment(flagsStringStream.str());
    }

    fs << "flags" << s.flag;

    fs << "fisheye_model" << s.useFisheye;

    fs << "camera_matrix" << cameraMatrix;
    fs << "distortion_coefficients" << distCoeffs;

    fs << "avg_reprojection_error" << totalAvgErr;
    if (s.writeExtrinsics && !reprojErrs.empty())
        fs << "per_view_reprojection_errors" << Mat(reprojErrs);

    if(s.writeExtrinsics && !rvecs.empty() && !tvecs.empty() )
    {
        CV_Assert(rvecs[0].type() == tvecs[0].type());
        Mat bigmat((int)rvecs.size(), 6, CV_MAKETYPE(rvecs[0].type(), 1));
        bool needReshapeR = rvecs[0].depth() != 1 ? true : false;
        bool needReshapeT = tvecs[0].depth() != 1 ? true : false;

        for( size_t i = 0; i < rvecs.size(); i++ )
        {
            Mat r = bigmat(Range(int(i), int(i+1)), Range(0,3));
            Mat t = bigmat(Range(int(i), int(i+1)), Range(3,6));

            if(needReshapeR)
                rvecs[i].reshape(1, 1).copyTo(r);
            else
            {
                //*.t() is MatExpr (not Mat) so we can use assignment operator
                CV_Assert(rvecs[i].rows == 3 && rvecs[i].cols == 1);
                r = rvecs[i].t();
            }

            if(needReshapeT)
                tvecs[i].reshape(1, 1).copyTo(t);
            else
            {
                CV_Assert(tvecs[i].rows == 3 && tvecs[i].cols == 1);
                t = tvecs[i].t();
            }
        }
        fs.writeComment("a set of 6-tuples (rotation vector + translation vector) for each view");
        fs << "extrinsic_parameters" << bigmat;
    }

    if(s.writePoints && !imagePoints.empty() )
    {
        Mat imagePtMat((int)imagePoints.size(), (int)imagePoints[0].size(), CV_32FC2);
        for( size_t i = 0; i < imagePoints.size(); i++ )
        {
            Mat r = imagePtMat.row(int(i)).reshape(2, imagePtMat.cols);
            Mat imgpti(imagePoints[i]);
            imgpti.copyTo(r);
        }
        fs << "image_points" << imagePtMat;
    }

    if( s.writeGrid && !newObjPoints.empty() )
    {
        fs << "grid_points" << newObjPoints;
    }
}

//! [run_and_save]
bool runCalibrationAndSave(Settings& s, Size imageSize, Mat& cameraMatrix, Mat& distCoeffs,
                           vector<vector<Point2f> > imagePoints, float grid_width, bool release_object)
{
    vector<Mat> rvecs, tvecs;
    vector<float> reprojErrs;
    double totalAvgErr = 0;
    vector<Point3f> newObjPoints;

    bool ok = runCalibration(s, imageSize, cameraMatrix, distCoeffs, imagePoints, rvecs, tvecs, reprojErrs,
                             totalAvgErr, newObjPoints, grid_width, release_object);
    cout << (ok ? "Calibration succeeded" : "Calibration failed")
         << ". avg re projection error = " << totalAvgErr << endl;

    if (ok)
        saveCameraParams(s, imageSize, cameraMatrix, distCoeffs, rvecs, tvecs, reprojErrs, imagePoints,
                         totalAvgErr, newObjPoints);
    return ok;
}
//! [run_and_save]

## Lab and independent exercises

1. Write code to save and read the calibration file.
2. Get a video of your own chessboard, calibrate your camera, and re-calculate the homography again. Then write code to rectify images in a video and show them side
   by side.
3. Using calibration images for the wide-angle Raspberry Pi camera on Matt's home robot, calibrate the camera then use the calibration to undistort the
   videos provided from the camera thus far.
4. Re-run your bird's-eye rectification from Lab 02 using the undistored images. Do you get a better result?

## Report

As always, turn in a report on your experiments and results by next lab.
